## IMAGE PRE PROCESSING
### 1. Import The ImageDataGenerator Library

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 1. Configure ImageDataGenerator Class


In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.1,
zoom_range = 0.1,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

### 2. Apply ImageDataGenerator Functionality To Trainset And Testset

In [3]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/level/training',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/level/validation',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


## MODEL BUILDING
### 1. Importing The Model Building Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

## 2. Loading The Model

In [5]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/level/training'
valid_path = '/content/drive/MyDrive/level/validation'

### 3. Adding Flatten Layer

In [6]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 2s 0us/step


In [7]:
for layer in vgg16.layers:layer.trainable = False
folders = glob('/content/drive/MyDrive/level/training/*')
folders

['/content/drive/MyDrive/level/training/02-moderate',
 '/content/drive/MyDrive/level/training/03-severe',
 '/content/drive/MyDrive/level/training/01-minor']

In [8]:
x = Flatten()(vgg16.output)
len(folders)

3

### 4. Adding Output Layer

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

### 5. Creating A Model Object

In [10]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 6. Configure The Learning Process

In [12]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

### 7. Train The Model

In [13]:
r = model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
98/98 [==============================] - 407s 4s/step - loss: 1.2409 - accuracy: 0.5628 - val_loss: 1.2019 - val_accuracy: 0.5614
Epoch 2/5
98/98 [==============================] - 18s 179ms/step - loss: 0.7316 - accuracy: 0.7191 - val_loss: 0.9586 - val_accuracy: 0.6082
Epoch 3/5
98/98 [==============================] - 16s 164ms/step - loss: 0.5469 - accuracy: 0.7957 - val_loss: 1.0207 - val_accuracy: 0.6140
Epoch 4/5
98/98 [==============================] - 16s 167ms/step - loss: 0.4278 - accuracy: 0.8223 - val_loss: 1.6515 - val_accuracy: 0.5965
Epoch 5/5
98/98 [==============================] - 17s 177ms/step - loss: 0.4449 - accuracy: 0.8284 - val_loss: 1.2299 - val_accuracy: 0.6199


### 8. Save The Model

In [14]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/ibm project/Intelligent Vehicle Damage Assessment & Cost Estimator/MODEL/LEVEL.h5')

### 9. Test The Model

In [15]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [16]:
model = load_model('/content/drive/MyDrive/ibm project/Intelligent Vehicle Damage Assessment & Cost Estimator/MODEL/LEVEL.h5')

In [17]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  if(np.max(img)>1):
    img = img/255.0
    img = np.array([img])
    prediction = model.predict(img)
    label = ["minor","moderate","severe"]
    preds = label[np.argmax(prediction)]
    return preds

In [18]:
data = "/content/drive/MyDrive/level/training/01-minor/0007.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 0s 157ms/step
minor
